# 실습 03: PaddleOCR 기본 사용법

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab03_PaddleOCR.ipynb)

## 🎯 학습 목표
- PaddleOCR 설치 및 기본 사용법
- 한글/영어 혼합 문서 인식
- 결과 시각화

## ⏱️ 소요 시간: 20분
## 📊 난이도: ⭐⭐☆☆☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# PaddleOCR 설치
!pip install -q paddlepaddle paddleocr
print("✅ PaddleOCR 설치 완료!")


In [ ]:
# 샘플 이미지 생성
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 한글 폰트 로드 (Windows/Linux 호환)
try:
    font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 20)
except:
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 20)
    except:
        font = ImageFont.load_default()

img = Image.new('RGB', (600, 400), color='white')
draw = ImageDraw.Draw(img)

text = """삼성전자 주식회사
Samsung Electronics

품의서 (Purchase Request)

품의번호: PR-2025-0001
작성일자: 2025년 1월 15일
담당부서: 구매팀
"""

draw.text((30, 30), text, fill='black', font=font)
img.save('sample_doc.jpg')

print("✅ 샘플 이미지 생성 완료!")
img


In [ ]:
# PaddleOCR 실행
from paddleocr import PaddleOCR

# 한글+영어 OCR 초기화 (최신 API)
ocr = PaddleOCR(use_textline_orientation=True, lang='korean')

# OCR 실행 (최신 API: predict 사용)
result = ocr.predict('sample_doc.jpg')

print("\n📄 인식된 텍스트:\n")
print("="*50)

# OCRResult 객체에서 텍스트와 신뢰도 추출
ocr_result = result[0]
texts = ocr_result['rec_texts']
scores = ocr_result['rec_scores']

# 결과 출력
for text, score in zip(texts, scores):
    print(f"{text:30s} (신뢰도: {score:.2%})")

print("="*50)

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# 한글 폰트 로드 (이미지에 텍스트 표시용)
try:
    label_font = ImageFont.truetype("C:\\Windows\\Fonts\\malgun.ttf", 14)
except:
    try:
        label_font = ImageFont.truetype("/usr/share/fonts/truetype/nanum/NanumGothic.ttf", 14)
    except:
        label_font = ImageFont.load_default()

# 디버깅: 원본 이미지와 OCR 처리 이미지를 나란히 비교
original_img = cv2.imread('sample_doc.jpg')
original_img_rgb = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

# OCR 결과에서 실제 처리된 이미지 가져오기
ocr_result = result[0]
doc_preprocessor_res = ocr_result.get('doc_preprocessor_res', {})
ocr_processed_img = doc_preprocessor_res.get('output_img', None)

print("="*70)
print("🔍 이미지 크기 비교")
print("="*70)
print(f"원본 이미지 shape: {original_img_rgb.shape} (H, W, C)")
print(f"OCR 처리 이미지 shape: {ocr_processed_img.shape if ocr_processed_img is not None else 'None'}")

# 박스 좌표 확인
boxes = ocr_result['rec_polys']
texts = ocr_result['rec_texts']
scores = ocr_result['rec_scores']

print(f"\n📦 박스 3개 상세 좌표:")
for i in range(min(3, len(boxes))):
    print(f"\n[{i+1}] '{texts[i]}' (신뢰도: {scores[i]:.1%})")
    print(f"    {boxes[i].tolist()}")

# OCR이 처리한 이미지가 원본과 다른지 확인
if ocr_processed_img is not None:
    # OCR 처리 이미지를 RGB로 변환 (시각화용)
    if len(ocr_processed_img.shape) == 2:
        ocr_vis = cv2.cvtColor(ocr_processed_img, cv2.COLOR_GRAY2RGB)
    else:
        ocr_vis = ocr_processed_img.copy()
    
    # OCR 처리 이미지에 박스 그리기 (원본 좌표 그대로)
    pil_ocr = Image.fromarray(ocr_vis)
    draw_ocr = ImageDraw.Draw(pil_ocr)
    
    for box, text, score in zip(boxes, texts, scores):
        if score < 0.5:
            continue
        points = [(int(p[0]), int(p[1])) for p in box.tolist()]
        color = (0, 255, 0) if score > 0.8 else (255, 165, 0)
        draw_ocr.polygon(points, outline=color, width=2)
        
        # 신뢰도 표시 추가 - 한글 폰트 지정!
        text_x = int(box[:, 0].min())
        text_y = int(box[:, 1].min()) - 15
        draw_ocr.text((text_x, max(0, text_y)), f"{text[:15]} {score:.0%}", fill=color, font=label_font)
    
    # 비교 시각화
    fig, axes = plt.subplots(1, 2, figsize=(20, 10))
    
    axes[0].imshow(original_img_rgb)
    axes[0].set_title(f'원본 이미지\n{original_img_rgb.shape[1]}x{original_img_rgb.shape[0]}px', fontsize=14)
    axes[0].axis('off')
    
    axes[1].imshow(pil_ocr)
    axes[1].set_title(f'OCR이 처리한 이미지 + 박스 (스케일 없음)\n{ocr_vis.shape[1]}x{ocr_vis.shape[0]}px', fontsize=14)
    axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n💡 OCR 처리 이미지에 박스가 정확히 맞는지 확인해보세요!")
    print(f"   만약 오른쪽 이미지에서 박스가 정확하다면,")
    print(f"   스케일 비율 = 원본크기 / OCR처리크기 로 계산해야 합니다.")
    
    # 정확한 스케일 계산
    scale_x = original_img_rgb.shape[1] / ocr_vis.shape[1]
    scale_y = original_img_rgb.shape[0] / ocr_vis.shape[0]
    print(f"\n🔧 정확한 스케일: X={scale_x:.3f}, Y={scale_y:.3f}")
else:
    print("⚠️ OCR 처리 이미지를 찾을 수 없습니다.")

print("="*70)